In [1]:
import os
os.chdir('..//')

In [2]:
!pwd

/home/debasish/home/Indian-Coin-Detection


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_dir: Path
    best_weight_path: Path
    last_weight_path: Path
    yolo_config_file_path: Path
    params_epochs: int
    params_image_size: list

In [4]:
from Indian_Coin_Detection.constants import *
from Indian_Coin_Detection.utils.common import read_yaml,create_directories
import os
import pathlib
import shutil
from ultralytics import YOLO

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params # all the parameters present inside params.yaml file
        yolo_config_file_path_ = self.config.training.yolo_config_file_path
        create_directories([
            Path(training.root_dir)
        ])
        create_directories([training.trained_model_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_dir=Path(training.trained_model_dir),
            best_weight_path=Path(training.best_weight_path),
            last_weight_path=Path(training.last_weight_path),
            yolo_config_file_path=Path(yolo_config_file_path_),
            params_epochs=params.EPOCHS,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    @staticmethod
    def save_weights(temp_weights_path: str,store_weights_path: Path):
    
        shutil.copyfile(temp_weights_path, store_weights_path)

    def train(self):
        model = YOLO("yolov8n.yaml")
        #model = YOLO("yolov8n.pt")

        results = model.train(data=self.config.yolo_config_file_path, 
                              epochs = self.config.params_epochs,
                              imgsz = self.config.params_image_size)
        
        temp_best_weights_path = "/home/debasish/home/Indian-Coin-Detection/runs/detect/train2/weights/best.pt"
        temp_last_weights_path = "/home/debasish/home/Indian-Coin-Detection/runs/detect/train2/weights/last.pt"
        
        self.save_weights(temp_best_weights_path,self.config.best_weight_path)
        self.save_weights(temp_last_weights_path,self.config.last_weight_path)

In [7]:
try:
    config = ConfigurationManager()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()
    
except Exception as e:
    raise e

[2023-09-06 01:53:52,314: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-06 01:53:52,322: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-06 01:53:52,323: INFO: common: created directory at: artifacts]
[2023-09-06 01:53:52,324: INFO: common: created directory at: artifacts/training]
[2023-09-06 01:53:52,326: INFO: common: created directory at: artifacts/training/model]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128